# 8.2 Processes

When we run a program, we assume that our program is the only on running on that system and that we have exclusive use of the processor and the memory. The processor seems to execute the instructions in our program without interruption, and the data and code of our program seem to be the only objects in the system's memory. This illusion is provided by the idea of a process. The definition of a process is an instance of a program in execution. Each program in the system runs in the context of some process, The context consists of the program state (i.e. program code and data in memory, stack, regusters etc.).  
Each time the user runs a program by typing the name of an executable object into the shell, the shell creates a new process and runs the executable file in the context of this new process. Application programs can create new processes and run their own code or other applications in the context of this new process.  
Processes provide some key abstractions.  
1. An independent logical control flow that provides the illusion of exclusive processor use
2. A private address space that provides the illusion of exclusive memory.

## 8.2.1 Logical Control Flow

If we were to use a debugger to do a step by step analysis of the program, we would observe a series of PC values that corresponded exclusively to the instructions ccontained in the program file. This sequence of PC values is known as the logical flow.  
Processes take turns using the processor. Each process executes a portion of its flow and is preempted while other processes take their turns. If we were to measure the time taken during execution, we would notice that the CPU appears to stall between some of the executions in our program.

## 8.2.2 Concurrent Flows

A logical flow whose execution overlaps in time with another flow is called a concurrent flow, and the two folws are said to run concurrently. Concurrency is the idea that multiple flows can execut concurrently. The notion of a process taking turns with other process is known as multitasking. Each time period that a process eecutes a portion of its flow is called a time slice. Multitasking is also referred to as time slicing. If two flows are running concurrently on different processors, they are known as parallel flows, and have parallel execution.

## 8.2.3 Private Address Space

A prrocess provides each program with the illusion that it has exclusive use of the system's address space. On a machine with n-bit addressesm the total adddress space is 2^n addresses. A process provides each program with its own private address space. The space is private in that it cannot be read or written by other processes. 
The bottom part of the address space is reserved for the user program, with the code, the data, the stack, and the heap. The code always begins at 0x400000. The top portion is reserved for the kernel, which contains the code, data, and stack the kernel uses when it executes instructions on behalf of the process.

## 8.2.4 User and Kernel Modes

Processors commonly carry a mode bit that characterizes the priveleges that the process enjoys. When the mode bit is set, the process is running in kernel mode. A process in kernel mode can execute any instructionand access any memory location in the system. When the mode bit is not set, the process is running in user mode. A process in user mode is not allowed to execute priveleged instructions. User programs must access kernel code via the system call interface.  

## 8.2.6 Context Switches

The OS implements multitasking using a form of exceptinoal control flow known as a context switch. It is built upon the exception mechanism discussed earlier.  
The kernel maintains a context for each process, and the context is the state that the kernel needs to restart a process. During execution, the kernel can preempt ehe current process and restart a previously preempted process. This decision is known as scheduling and is handled by the scheduler.  
After the kernel has selected a new process to run, it preempts the current process and transfers control to the new process using a context switch that saves the context of the current process, restores the context of the previously preempted process and passes control to this new process.

A context switch could occur during a syscall, If a syscall blocks since it is waiting for some event to occur, the kernel can preempt the current process and switch to another process.  
A context switch can also occur as a result of an interrupt. If a timer interrupt occurs, the kernel can decide if it has to switch to another process.

# 8.3 System Call Error Handling

When system level functions encounter an error, they typically return 1 and set the global variable errno to indicate what went wrong. We can define error handling functions to help us find out what went wrong. C errors aren't always the best T_T.

# 8.4 Process Control

Unix provides a lot of syscalls for manipulating processes from C programs.

## 8.4.1- 8.4.2

Creating Process IDs - getpid (returns PID of calling process), getppid (returns PID of the parent, process that created calling process)

Creating and Terminating processes - process is either in one of three states, running (executing, or waiting to be executed), stopped (execution is suspended and will not be scheduled), or terminated (oricess stopped perminantly . by receiving a signal to terminate process, returning from main routine, calling exit function)

A parent process can create a new child process by calling the fork function. The new child process is almost identical to the parent process. The child gets an identical copy of the parent's user level data. The child can read and write any files opened by the parent. They do have different PIDs though.  
The fork function is called once, but it returns twice, once in the calling function, and again in the child process. The fork returns the PID of the child in the parent, and returns the value 0 in the child process. Since the PID of the child is nonzero, the return value provides information on whether the program is executing in the parent or the child.  
There are, however, some subtleties:  
1. Call once, return twice (exaplained above).  
2. Concurrent execution: The parent and the child are separate processes that run concurrently. How their control flows are handled are determined by the kernel arbitrarily (could be that the parent is first or the other way around).  
3. Duplicate but separate address space: If we halted the parent and the child after the fork, we would notice that the address space of each process is identical, but since they are separate processes, they have their own private address spaces. Any changes that a parent or child makes are not reflected in the memory of the other process.  
4. Shared files: When we run a program that has used fork, both the parent and child print their output on the screen. This is because the child inherits all of the parent's files.

It is often helpful to sketch a process graph. Earch vertex in this graph corresponds to the execution of the program statement.  
For a program running on a single processor, any topological sort of the process graphs represents a total ordering of statements in the program. 

## 8.4.3 Reaping Child Processes

When a process terminates, thge kernel does not remove it from the system. It is instead kept until it is reaped by the parent. When the parent reaps the child, the kernel passes the child's exit status to the parent and discards the terminated child process, which is called a zombie.  
When a parent is terminated, the kernl arranges for the init process to become the adopted parent of any orphaned children. The init process is the ancestor of every process. If a parent process terminates without reaping the children, the init process will reap them. Long running programs should always reap their zombie children. A process waits for its children to terminate by calling the waitpid function.

## 8.4.4 Putting Processes to sleep

The sleep function suspends a process for a specified period of time. Pause also puts the function to sleep until a signal is received by the process

## 8.4.5 Loading and Running Programs

The execve function loads and runs the executable object file. 

## 8.4.6 Using fork and execve to run programs

Programs such as unix shells and web servers use fork and execve very often. A shell is an interactive application program that runs other programs on behalf of the user. A shell performs a sequence of read/evaluate steps and then terminates. The read steps reads a command line from the user, and the evaluate step parses the command line and runs programs on behalf of the user. 

# 8.5 Signals

A signal is a software level exception that allows processes and the kernel to interrupt other processes. Mor precisely, it is a small message that notifies a process that an event of some type occured in the system.  
Low level hardware exceptions are processed by the kernel's exception handlers and are not normally visible to user processes. Signals provide a mechanism for exposing hardware level exceptions. 

## 8.5.1 Signal Terminology

The kernel sends a signal to a destination process by updating some state in the context of destination process. The signal is delivered for one of two reasons
1. The kernel has detected a system event
2. A process has invoked the kill function to request the kernel to send a signal to the destination process  
  
  
A destination process receives a signal when it is forced by the kernel to react in some way to the delivery of the signal. The process can ignore, terminate, or catch the signal by using a signal handler.

A signal that has been sent but not yet received is called a pending signal. At any point, there can be at most one pending signal of a particular type.  
A process can selectively block the receipt of certain signals. When a signal is blocked, it can be delivered but the resulting signal will not be received until the process unblocks the signal.  
A pending signal is received at most once. For each process, the kernel maintains the set of pending signals, and the set of blocked signals.

## 8.5.2 Sending Signals

Unix systems provide a number of mechanisms for sending signals to processes, all of which rely on the notion of the process group.  
Every process belongs to exactly one process group, which is identified by the process group ID. By default, a child process belongs to the same process group as its parent. A parent can change the pocess group of itself or another process by using the setpgrid function.  
Unix systems use the idea of a job to represent the processes that are created as a result of running a single command line. At any point in time, there is at most one foreground job and zero or more background jobs.

## 8.5.3 Receiving Signals

When the kernel switches a process from kernel to user mode, it checks the set of unblocked pending signals. If this set is empty, it goes to the next instruction. Otherwise, the kernel chooses a signal in the set and forces the process to receive the signal. The reception of this signal triggers some action by the process. Once the process completes the action, the control passes back to the next instruction in the control flow. Each signal type has a predefined default action, which is one of the following: 
1. Terminates process
2. Terminates process and dumps core
3. Stops process until restarted
4. Process ignores signal

The signal function can change the action associated with a signal in one of three ways:
1. If a signal is SIG_IGN, the signals of type signum are ignored
2. If a signal is SIG_DFL, the action for signals of type signum reverts to the default action
3. Otherwise,the handler is the address of a user defined function that will be called whenever the process receives a signal of type signum. Changing the default action by passing the address of a handler is called installing the handler. Invoking this handler is called catching the signal. Execution of this handler is called handling the signal

When a process catches a specific type of a signal handled by a specific handler, it is invoked by a single integer argument, and allows the same handler function to catch different types of signals. When the handler executes its return statement, control usually passes back to the instruction in the flow where the processs was interrupted by the signal. In some systems, interrupted syscalls return with an error.  
Signal handlers can be interrupted by other handlers.

## 8.5.4 Blocking and Unblocking Signals

Linux provides implicit and explicit methods for blocking signals:

Implicit blocking mechanism: By defualt, the kernel blocks any pending signals of they type currently being processed.  
Explicit blocking mechanism: Applications can explicitly block and unblock selected signals

## 8.5.5 Writing Signal Handlers

Handlers have several different attributes that make them difficult to think about
1. Handlers run concurrently with the main program and share the same global variables and thus can interfere with the main program and with other handlers.
2. The rules for how and when signals are received is often conterintuitive
3. Different systems can have different signal handling semantics

Rules for safe signal handling:
1. Keep handlers as simple as possible.
2. Call only async-signal-safe functions in your handlers. Safe functions has the property that it can be safely called from a signal handler because it is reentrant (accesses only local variables) or because it cannot be interrupted by a signal handler. The only way to generate output from a signal handler is to use write.
3. save and restore errno. Many safe functions set errno when they return with an error. Calling such functions inside the handler might interfere with other functions that rely on errno. The workaround is to save errno locally and restore it when the handler returns.  
4. Protect accesses to shared global data structures by blocking all signals. If a handler shares a global data structure with the main program or with other handlers, then your handlers and the main program should block all signals from accessing that data structure. The reason is that accessing the data structure from the main program requires a sequence of instructions. If this instruction sequence is interrupted by a handler that access the data structre, the handler might find the data structure in an unpredictable state. Blocking signals guarantees that the handler will find the data structure in a consistent state.
5. Declate global variables with volatile. This forces the compiler to read the value from memory every time it is referenced to force it to read the newest value rather than reading the cached value.
6. Declare flags with sig_atomic_t. Commonly, the handler records receipt of the signal by writing to a global flag. The main program reads the flag, responds to the signal and clears the flag. Using sig_atomic_t guarantees that reads and writes are atomic (uninterruptable). Thus, you can safely read and write to sig_atomic_t without blocking signals.  
Signals cannot be used to count events in other processes. To do so, you have to get your handler to reap as many zombie children as possible each time it is invoked.

Every machine has different signal handling semantics.  
1. The signal function has different semantics, some unix systems restore the action for a signal to its default after it has been caught by a handler. The handler must explicitly reinstall itself each time it runs.
2. Syscalls can be interrupted. Syscalls such as read, wait, accept that can block the process for a long time are known as slow system calls. When these syscalls are interrupted when a handler catche a signal does not resume when the handler returns but insteads returns to the user with an error condition. The programmer must manually restart the syscalls in this case.  
  
To deal with these issues, the developer can use the sigaction function which allows users to specify the signal handling semantics they want when the handler is installed.  
However, this is unwieldy, because it requires the user to set the entries of a complicated data structure. A cleaner approach is to define a wrapper function that calls sigaction. This wrapper installs a signal handler with the following semantics.
1. Only signals of the type being processed are blocked . 
2. signals are not queued
3. Interrupted syscalls are restarted whenever possible
4. Ths signal handler is installed and remains installed unless it is called with a handler argument of ignore or default handling

## 8.5.6 Synchronizing flows to avoid nasty concurrency bugs

A big question is how to program concurrent flows such that you can synchronize these flows to provide the largest set of feasible interleavings such that each of the interleavings provides a correct answer.   
After a parent creates a new child process, it adds the child to the job list. After the terminated process has been reaped, the parent deletes the child from the job list  
This seems correct but the following could happen:  
1. The parent executes a fork and the kernel schedules the child to run instead of the parent
2. Before the parent s able to run, the child terminates and becomes a zombie and delivers a SIGCHLD signal to the parent
3. When the parent is runnable again, the kernel notices this pending signal and causes it to be received by the running handler
4. The handler reaps the terminated child and calls deletejob, which does nothing since the child has not been added by the parent
5. The handler completes and the kernel runs the parent which returns the fork and adds the nonexistent child to the job  
  
It is possible for deletejob to be called before addjob, which causes an incorrect entry in the joblist.  
There is also a scenario in which it is run correctly, where the parent is scheduled to run when the fork call returns and will correctly add the child job to the list.  
This is a synchronization error called a race.  
One way to solve the above answer is to block all SIGCHLD signals before the call to fork and unblocking them after we have called addjob, and guaranteeing tha tthe child will be reaped after it is added to the job list.   
However, the parent will inherit the blocked set of signals from the parent, so we have to unblock this signal in the child.

## 8.5.7 Explicitly waiting for signals

Someitimes, a program needs to wait for a certain signal handler to run. For example, in a linux shell, whenever a command is run, it must wait for the job to be terminated and reaped before accepting the next command.  
One way you can handle this is to have a while loop that stops whenever the PID is nonzero, which means the process has been reaped. However, this causes a huge amount of resources to be used, and is super wasteful. If we insert a pause in the spin loop, that will cause a race condition. If we use a sleep, that is very inefficient. The sigsuspend function replaces the blocked set with a mask and then suspends the process until the receipt of a signal whose action is either to run a handler or terminate the process. It is atomic. 

# 8.6 Nonlocal Jumps

C procides a user level ECF that transfers control from one function to another currently executing function without going through the call return sequence., which is called a nonlocal jump. An important application is to permit an immediate return from a deeply nested function call. You can reference a local error handler. Another important application is to branch out of a signal handler to a specific code location, rather than returning to the instruction that was interrupted by the arrival of the signal.